# Определение возраста покупателей

**Описание проекта:**

Сетевой супермаркет внедряет систему компьютерного зрения для обработки фотографий покупателей. 
Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:
* Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
* Контролировать добросовестность кассиров при продаже алкоголя.

**Цель проекта:**

Модель, которая по фотографии определит приблизительный возраст человека.

**Критерии качества:**

Метрика МАЕ меньше 8

## Исследовательский анализ данных

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os
import pandas as pd
from tensorflow.keras.applications.resnet import ResNet50

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')

In [ ]:
labels.shape

In [ ]:
labels

Посмотрим на распределение возрастов в выборке

In [ ]:
#sns.set(font_scale = 1.5)
plt.figure(figsize = (30,10))
sns.countplot(labels['real_age'])
plt.xticks(rotation=90)

Видим, что людей сстарше 60 лет гораздо меньше чем более молодых. Людей в возрасте от 14-40 лет больше всего - больше 100 человек для каждого возраста в этом диапазоне.

In [ ]:
labels.describe()

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
datagen_flow = datagen.flow_from_directory(
    '/datasets/faces/',

    target_size=(150, 150),
    batch_size=16,
    class_mode='sparse',
    subset='training',
    seed=12345)


features, target = next(datagen_flow)



In [ ]:
# выводим 16 изображений
fig = plt.figure(figsize=(10,10))
for i in range(16):
    indexes = datagen_flow.index_array[:16]
    fig.add_subplot(4, 4, i+1).set_title(f'Возраст: {labels.real_age.iloc[indexes[i]]}')
    plt.imshow(features[i])
	# для компактности удаляем оси и прижимаем изображения друг к другу
    plt.xticks([])
    plt.yticks([])
    #plt
    plt.tight_layout()

Вывод по результатам исследования:

Судя по распределению возраста в имеющихся данных лучше всего должны будут быть распознаны лица людей в возрасте от 20 до 41 года, так же достаточно много данных для детей возрастом до 5 лет, а вот с рапознаванием возраста людей страше 70 лет достаточно мало данных и могут быть трудности. Так же есть фото, например, с жевательной резинкой, которая закрывает часть лица - такие фото ухудшат обучаемость. Есть фото повернутые, есть черно-белые.

## Обучение модели

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    train_datagen = ImageDataGenerator(
        validation_split=0.25,
        horizontal_flip=True,
        rescale=1./255)
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        seed=12345)
    return train_gen_flow


In [ ]:
def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')
    test_datagen = ImageDataGenerator(
        validation_split=0.25,
        rescale=1./255)
    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='validation',
        seed=12345)
    return test_gen_flow

In [ ]:
def create_model(input_shape):
    backbone = ResNet50(weights='imagenet', 
                        input_shape=input_shape,
                        include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))
    optimizer = Adam(lr=0.0005)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

In [ ]:
def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)
    return model

Вычисление производилось на сторонних мощностях. Ниже приведен результат вычисления. Поставленная цель достигнута.

Результат обучения:

```python
Train for 356 steps, validate for 119 steps
Epoch 1/20
2021-07-29 13:59:50.556586: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-07-29 13:59:50.842363: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
356/356 - 46s - loss: 209.9321 - mae: 10.9374 - val_loss: 629.9441 - val_mae: 20.1435
Epoch 2/20
356/356 - 37s - loss: 135.2814 - mae: 8.8849 - val_loss: 225.1827 - val_mae: 11.8003
Epoch 3/20
356/356 - 37s - loss: 111.6643 - mae: 8.0298 - val_loss: 130.5670 - val_mae: 9.0427
Epoch 4/20
356/356 - 38s - loss: 91.3143 - mae: 7.3465 - val_loss: 120.7078 - val_mae: 8.2999
Epoch 5/20
356/356 - 38s - loss: 81.2874 - mae: 6.8389 - val_loss: 157.4068 - val_mae: 9.2262
Epoch 6/20
356/356 - 38s - loss: 71.0115 - mae: 6.4768 - val_loss: 127.3828 - val_mae: 8.5481
Epoch 7/20
356/356 - 38s - loss: 54.9083 - mae: 5.7000 - val_loss: 128.8676 - val_mae: 8.6193
Epoch 8/20
356/356 - 38s - loss: 47.8948 - mae: 5.3213 - val_loss: 93.2415 - val_mae: 7.2711
Epoch 9/20
356/356 - 38s - loss: 39.8999 - mae: 4.8333 - val_loss: 96.5398 - val_mae: 7.4227
Epoch 10/20
356/356 - 38s - loss: 34.2201 - mae: 4.5080 - val_loss: 99.8048 - val_mae: 7.3277
Epoch 11/20
356/356 - 38s - loss: 31.1639 - mae: 4.2972 - val_loss: 102.3363 - val_mae: 7.6928
Epoch 12/20
356/356 - 38s - loss: 27.1053 - mae: 4.0281 - val_loss: 98.4520 - val_mae: 7.5206
Epoch 13/20
356/356 - 38s - loss: 24.7616 - mae: 3.8111 - val_loss: 94.8010 - val_mae: 7.4353
Epoch 14/20
356/356 - 38s - loss: 25.8449 - mae: 3.9135 - val_loss: 93.6739 - val_mae: 7.3047
Epoch 15/20
356/356 - 38s - loss: 24.9931 - mae: 3.8217 - val_loss: 87.3194 - val_mae: 7.0132
Epoch 16/20
356/356 - 38s - loss: 19.8076 - mae: 3.4278 - val_loss: 78.0335 - val_mae: 6.6319
Epoch 17/20
356/356 - 38s - loss: 18.6851 - mae: 3.3224 - val_loss: 91.9763 - val_mae: 7.1595
Epoch 18/20
356/356 - 38s - loss: 17.8242 - mae: 3.2381 - val_loss: 82.7343 - val_mae: 6.8413
Epoch 19/20
356/356 - 38s - loss: 15.6390 - mae: 3.0600 - val_loss: 92.9889 - val_mae: 7.4051
Epoch 20/20
356/356 - 38s - loss: 15.6837 - mae: 3.0655 - val_loss: 82.1044 - val_mae: 7.0108
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
119/119 - 9s - loss: 82.1044 - mae: 7.0108
Test MAE: 7.0108
```